# Pre-processing the data - output intermidiate gridded
This notebook follows on from `transform_and_preprocess_spice.ipynb`, and aims to output the intermediate MOGREPS-G data for the UK in gridded form in a single file, which was proving problematic intially in other notebook due to resource issues.

In [1]:
import pathlib
import datetime
import functools
import os

In [2]:
import numpy

In [3]:
import pandas

In [4]:
import xarray
import iris
import iris.quickplot
import iris.coord_categorisation

In [5]:
import matplotlib.pyplot

# Set parameters for notebook
Set the paths and lists of things to process

In [6]:
project_name = 'precip_rediagnosis'
mogreps_g_name = 'mogreps-g'
ilab_project_dir = pathlib.Path('/project/informatics_lab/')
output_dir =  pathlib.Path('/scratch')/ os.environ['USER'] / project_name

In [7]:
root_data_dir = ilab_project_dir / project_name
mogreps_g_data_dir = root_data_dir / mogreps_g_name
radar_data_dir = root_data_dir / 'radar'

In [8]:
date_fname_template = '{start.year:04d}{start.month:02d}{start.day:02d}T{start.hour:02d}{start.minute:02d}Z_{end.year:04d}{end.month:02d}{end.day:02d}T{end.hour:02d}{end.minute:02d}Z'
fname_extension_grid = '.nc'
fname_extension_tabular = '.csv'
leadtime_template = '{lt:03d}H'
mogreps_g_tab_fname_template = 'prd_mogreps_g_' + leadtime_template + '_' + date_fname_template + fname_extension_tabular
mogreps_g_grid_fname_template = 'prd_mogreps_g_' + leadtime_template + '_' + date_fname_template + fname_extension_grid
radar_tab_fname_template = 'prd_radar_' + date_fname_template + fname_extension_tabular
radar_grid_fname_template = 'prd_radar_' + date_fname_template + fname_extension_grid
output_fname_template = 'prd_merged_' + leadtime_template + '_' + date_fname_template + fname_extension_tabular

In [9]:
variables_single_level = [
    "cloud_amount_of_total_cloud",
    "rainfall_accumulation-PT03H",
    "snowfall_accumulation-PT03H",
    "rainfall_rate",
    "snowfall_rate",
    "height_of_orography",
    "pressure_at_mean_sea_level",
]

variables_height_levels = [
    "cloud_amount_on_height_levels",
    "pressure_on_height_levels",
    "temperature_on_height_levels",
    "relative_humidity_on_height_levels",
    "wind_direction_on_height_levels",
    "wind_speed_on_height_levels",
    
]

In [10]:
num_periods = 10
start_ref_time = datetime.datetime(2020,2,14,12)
forecast_ref_time_range = [start_ref_time + datetime.timedelta(hours=6)*i1 for i1 in range(num_periods)]
leadtime_hours = 15
realizations_list = list(range(35))

In [11]:
dataset = 'mogreps-g'
subset = 'lev1'
forecast_ref_template = '{frt.year:04d}{frt.month:02d}{frt.day:02d}T{frt.hour:02d}00Z.nc.file'
fname_template = '{vt.year:04d}{vt.month:02d}{vt.day:02d}T{vt.hour:02d}00Z-PT{lead_time:04d}H00M-{var_name}.nc'

In [12]:
variables_to_extract = variables_height_levels + variables_single_level

In [13]:
path_lists_vars = {
    var_name: [f1 for f1 in mogreps_g_data_dir.iterdir() if var_name in str(f1)]
    for var_name in variables_to_extract
}


In [14]:
uk_bounds={'latitude':(50,58), 'longitude': (-6,2)}
xarray_select_uk = {k1: slice(*v1) for k1,v1 in uk_bounds.items()}

In [15]:
def cftime_to_datetime(input_cft):
    return datetime.datetime(input_cft.year,
                             input_cft.month,
                             input_cft.day,
                             input_cft.hour,
                             input_cft.minute,
                             input_cft.second,
                            )

## Create a dataset from MOGREPS-G data
Information on Met Office Ensmble forecasts - https://www.metoffice.gov.uk/research/weather/ensemble-forecasting#
Paper - https://www.metoffice.gov.uk/research/weather/ensemble-forecasting 

### Get the mapping of variable names 
Load some files and get the actual variable names.

In [16]:
fcst_ref_time = forecast_ref_time_range[0]
real1 = realizations_list[10]
validity_time = fcst_ref_time + datetime.timedelta(hours=leadtime_hours)

In [17]:
%%time
# load a cube for each variable in iris to get the actual variable name, and populate dictionary mapping from the var name in the file name to the variable as loaded into iris/xarray
file_to_var_mapping = {
    var_file_name: iris.load_cube(str(mogreps_g_data_dir / fname_template.format(vt=validity_time,
                                                                                 lead_time=leadtime_hours,
                                                                                 var_name=var_file_name))).name()
    for var_file_name in variables_single_level + variables_height_levels}
file_to_var_mapping

CPU times: user 850 ms, sys: 88.5 ms, total: 939 ms
Wall time: 2.07 s


{'cloud_amount_of_total_cloud': 'cloud_area_fraction',
 'rainfall_accumulation-PT03H': 'thickness_of_rainfall_amount',
 'snowfall_accumulation-PT03H': 'lwe_thickness_of_snowfall_amount',
 'rainfall_rate': 'rainfall_rate',
 'snowfall_rate': 'lwe_snowfall_rate',
 'height_of_orography': 'surface_altitude',
 'pressure_at_mean_sea_level': 'air_pressure_at_sea_level',
 'cloud_amount_on_height_levels': 'cloud_volume_fraction_in_atmosphere_layer',
 'pressure_on_height_levels': 'air_pressure',
 'temperature_on_height_levels': 'air_temperature',
 'relative_humidity_on_height_levels': 'relative_humidity',
 'wind_direction_on_height_levels': 'wind_from_direction',
 'wind_speed_on_height_levels': 'wind_speed'}

In [18]:
heights = iris.load_cube(str(mogreps_g_data_dir / fname_template.format(vt=validity_time,
                                                                                 lead_time=leadtime_hours,
                                                                                 var_name=variables_height_levels[0]))).coord('height').points

In [19]:
merge_coords = ['latitude', 'longitude', 'time', 'realization']

In [20]:
single_level_var_mappings = {v1: file_to_var_mapping[v1] for v1 in variables_single_level}
height_level_var_mappings = {v1: file_to_var_mapping[v1] for v1 in variables_height_levels}

In [21]:
def load_ds(ds_path, selected_bounds):
    try:
        subset1 = dict(selected_bounds)
        subset1['bnds'] = 0
        single_level_ds = xarray.load_dataset(ds_path).sel(**subset1)
    except KeyError as e1:
        single_level_ds = None
    return single_level_ds

In [22]:
%%time
# gridded_data_list = []
for fcst_ref_time in forecast_ref_time_range:
    print(fcst_ref_time)
    validity_time = fcst_ref_time + datetime.timedelta(hours=leadtime_hours)
    single_level_ds = xarray.merge([load_ds(ds_path= mogreps_g_data_dir / fname_template.format(vt=validity_time,
                                                                                                lead_time=leadtime_hours,
                                                                                                var_name=var1),
                                            selected_bounds=xarray_select_uk,
                                           )
                                    for var1 in variables_single_level]
                                  )

    height_levels_ds = xarray.merge([load_ds(ds_path=mogreps_g_data_dir / fname_template.format(vt=validity_time,
                                                                                                lead_time=leadtime_hours,
                                                                                                var_name=var1),
                                             selected_bounds=xarray_select_uk,
                                            )
                                     for var1 in variables_height_levels])
    
    ts_mogg_ds1 = xarray.merge([height_levels_ds, single_level_ds])
    ts_mogg_ds1.to_netcdf(output_dir / (
        'prd_mg_ts_'+ f'{validity_time.year:04d}{validity_time.month:02d}{validity_time.day:02d}{validity_time.hour:02d}{validity_time.minute:02d}' 
        + fname_extension_grid)
    )
    # gridded_data_list += [xarray.merge([height_levels_ds, single_level_ds])]

2020-02-14 12:00:00
2020-02-14 18:00:00
2020-02-15 00:00:00
2020-02-15 06:00:00
2020-02-15 12:00:00
2020-02-15 18:00:00
2020-02-16 00:00:00
2020-02-16 06:00:00
2020-02-16 12:00:00
2020-02-16 18:00:00
CPU times: user 22min 11s, sys: 2min 37s, total: 24min 48s
Wall time: 33min 22s


In [23]:
merged_ds = xarray.concat([xarray.load_dataset(f1) for f1 in output_dir.iterdir() if 'prd_mg_ts' in str(f1)], dim='time')

In [24]:
start_dt = min(forecast_ref_time_range) + datetime.timedelta(hours=leadtime_hours)
end_dt = max(forecast_ref_time_range) + datetime.timedelta(hours=leadtime_hours)

In [25]:
merged_ds.to_netcdf(
    output_dir / mogreps_g_grid_fname_template.format(start=start_dt,
                                               end=end_dt,
                                               lt=leadtime_hours,
                                              )
)

/var/tmp/ipykernel_31413/3187837745.py:1: SerializationWarning: saving variable forecast_reference_time with floating point data as an integer dtype without any _FillValue to use for NaNs
  merged_ds.to_netcdf(
/var/tmp/ipykernel_31413/3187837745.py:1: SerializationWarning: saving variable time_bnds with floating point data as an integer dtype without any _FillValue to use for NaNs
  merged_ds.to_netcdf(
